## ROI selection reconstruction into a mask

From the nicely formatted _ROI.csv and _emb.csv files, we want to create an image mask of the same number of pixels as the source image, which will be our mask.

In [1]:
roi_csv_df = pd.read_csv(f"{}")
roi_mask_i = np.zeros((2, 3)) - 1

SyntaxError: f-string: empty expression not allowed (<ipython-input-1-4774ae010db9>, line 1)